In [4]:
import altair as alt
import streamlit as st
import twitter_api
# import predict_live_tweets
# import trading_algorithm
# import time_series_plot
import numpy as np
import pandas as pd
import torch
from torch import nn
import time
from transformers import BertModel, AdamW

class SentimentClassifier(nn.Module):

  def __init__(self, n_classes):
    super(SentimentClassifier, self).__init__()
    self.bert = BertModel.from_pretrained('bert-base-cased', return_dict=False)
    self.dropout = nn.Dropout(p=0.3)
    self.out = nn.Linear(self.bert.config.hidden_size, n_classes)

  def forward(self, input_ids, attention_mask):
    _, output = self.bert(input_ids=input_ids, attention_mask=attention_mask)
    output = self.dropout(output)
    return self.out(output)

KeyboardInterrupt: 

In [5]:
# sp500_stocks = ["$A", "$AAL", "$AAP", "$AAPL", "$ABBV", "$ABC", "$ABMD", "$ABT", "$ACN", "$ADBE", "$ADI", "$ADM", "$ADP", "$ADSK", "$AEE", "$AEP", "$AES", "$AFL", "$AIG", "$AIZ", "$AJG", "$AKAM", "$ALB", "$ALGN", "$ALK", "$ALL", "$ALLE", "$AMAT", "$AMCR", "$AMD", "$AME", "$AMGN", "$AMP", "$AMT", "$AMZN", "$ANET", "$ANSS", "$ANTM", "$AON", "$AOS", "$APA", "$APD", "$APH", "$APTV", "$ARE", "$ATO", "$ATVI", "$AVB", "$AVGO", "$AVY", "$AWK", "$AXP", "$AZO", "$BA", "$BAC", "$BAX", "$BBWI", "$BBY", "$BDX", "$BEN", "$BF.B", "$BIIB", "$BIO", "$BK", "$BKNG", "$BKR", "$BLK", "$BLL", "$BMY", "$BR", "$BRK.B", "$BRO", "$BSX", "$BWA", "$BXP", "$C", "$CAG", "$CAH", "$CARR", "$CAT", "$CB", "$CBOE", "$CBRE", "$CCI", "$CCL", "$CDAY", "$CDNS", "$CDW", "$CE", "$CEG", "$CERN", "$CF", "$CFG", "$CHD", "$CHRW", "$CHTR", "$CI", "$CINF", "$CL", "$CLX", "$CMA", "$CMCSA", "$CME", "$CMG", "$CMI", "$CMS", "$CNC", "$CNP", "$COF", "$COO", "$COP", "$COST", "$CPB", "$CPRT", "$CPT", "$CRL", "$CRM", "$CSCO", "$CSX", "$CTAS", "$CTLT", "$CTRA", "$CTSH", "$CTVA", "$CTXS", "$CVS", "$CVX", "$CZR", "$D", "$DAL", "$DD", "$DE", "$DFS", "$DG", "$DGX", "$DHI", "$DHR", "$DIS", "$DISH", "$DLR", "$DLTR", "$DOV", "$DOW", "$DPZ", "$DRE", "$DRI", "$DTE", "$DUK", "$DVA", "$DVN", "$DXC", "$DXCM", "$EA", "$EBAY", "$ECL", "$ED", "$EFX", "$EIX", "$EL", "$EMN", "$EMR", "$ENPH", "$EOG", "$EPAM", "$EQIX", "$EQR", "$ES", "$ESS", "$ETN", "$ETR", "$ETSY", "$EVRG", "$EW", "$EXC", "$EXPD", "$EXPE", "$EXR", "$F", "$FANG", "$FAST", "$FB", "$FBHS", "$FCX", "$FDS", "$FDX", "$FE", "$FFIV", "$FIS", "$FISV", "$FITB", "$FLT", "$FMC", "$FOX", "$FOXA", "$FRC", "$FRT", "$FTNT", "$FTV", "$GD", "$GE", "$GILD", "$GIS", "$GL", "$GLW", "$GM", "$GNRC", "$GOOG", "$GOOGL", "$GPC", "$GPN", "$GRMN", "$GS", "$GWW", "$HAL", "$HAS", "$HBAN", "$HCA", "$HD", "$HES", "$HIG", "$HII", "$HLT", "$HOLX", "$HON", "$HPE", "$HPQ", "$HRL", "$HSIC", "$HST", "$HSY", "$HUM", "$HWM", "$IBM", "$ICE", "$IDXX", "$IEX", "$IFF", "$ILMN", "$INCY", "$INTC", "$INTU", "$IP", "$IPG", "$IPGP", "$IQV", "$IR", "$IRM", "$ISRG", "$IT", "$ITW", "$IVZ",
#                 "$J", "$JBHT", "$JCI", "$JKHY", "$JNJ", "$JNPR", "$JPM", "$K", "$KEY", "$KEYS", "$KHC", "$KIM", "$KLAC", "$KMB", "$KMI", "$KMX", "$KO", "$KR", "$L", "$LDOS", "$LEN", "$LH", "$LHX", "$LIN", "$LKQ", "$LLY", "$LMT", "$LNC", "$LNT", "$LOW", "$LRCX", "$LUMN", "$LUV", "$LVS", "$LW", "$LYB", "$LYV", "$MA", "$MAA", "$MAR", "$MAS", "$MCD", "$MCHP", "$MCK", "$MCO", "$MDLZ", "$MDT", "$MET", "$MGM", "$MHK", "$MKC", "$MKTX", "$MLM", "$MMC", "$MMM", "$MNST", "$MO", "$MOH", "$MOS", "$MPC", "$MPWR", "$MRK", "$MRNA", "$MRO", "$MS", "$MSCI", "$MSFT", "$MSI", "$MTB", "$MTCH", "$MTD", "$MU", "$NCLH", "$NDAQ", "$NDSN", "$NEE", "$NEM", "$NFLX", "$NI", "$NKE", "$NLOK", "$NLSN", "$NOC", "$NOW", "$NRG", "$NSC", "$NTAP", "$NTRS", "$NUE", "$NVDA", "$NVR", "$NWL", "$NWS", "$NWSA", "$NXPI", "$O", "$ODFL", "$OGN", "$OKE", "$OMC", "$ORCL", "$ORLY", "$OTIS", "$OXY", "$PARA", "$PAYC", "$PAYX", "$PCAR", "$PEAK", "$PEG", "$PENN", "$PEP", "$PFE", "$PFG", "$PG", "$PGR", "$PH", "$PHM", "$PKG", "$PKI", "$PLD", "$PM", "$PNC", "$PNR", "$PNW", "$POOL", "$PPG", "$PPL", "$PRU", "$PSA", "$PSX", "$PTC", "$PVH", "$PWR", "$PXD", "$PYPL", "$QCOM", "$QRVO", "$RCL", "$RE", "$REG", "$REGN", "$RF", "$RHI", "$RJF", "$RL", "$RMD", "$ROK", "$ROL", "$ROP", "$ROST", "$RSG", "$RTX", "$SBAC", "$SBNY", "$SBUX", "$SCHW", "$SEDG", "$SEE", "$SHW", "$SIVB", "$SJM", "$SLB", "$SNA", "$SNPS", "$SO", "$SPG", "$SPGI", "$SRE", "$STE", "$STT", "$STX", "$STZ", "$SWK", "$SWKS", "$SYF", "$SYK", "$SYY", "$T", "$TAP", "$TDG", "$TDY", "$TECH", "$TEL", "$TER", "$TFC", "$TFX", "$TGT", "$TJX", "$TMO", "$TMUS", "$TPR", "$TRMB", "$TROW", "$TRV", "$TSCO", "$TSLA", "$TSN", "$TT", "$TTWO", "$TWTR", "$TXN", "$TXT", "$TYL", "$UA", "$UAA", "$UAL", "$UDR", "$UHS", "$ULTA", "$UNH", "$UNP", "$UPS", "$URI", "$USB", "$V", "$VFC", "$VLO", "$VMC", "$VNO", "$VRSK", "$VRSN", "$VRTX", "$VTR", "$VTRS", "$VZ", "$WAB", "$WAT", "$WBA", "$WBD", "$WDC", "$WEC", "$WELL", "$WFC", "$WHR", "$WM", "$WMB", "$WMT", "$WRB", "$WRK", "$WST", "$WTW", "$WY", "$WYNN", "$XEL", "$XOM", "$XRAY", "$XYL", "$YUM", "$ZBH", "$ZBRA", "$ZION", "$ZTS"]
sp500_stocks = ["$APPL","$META"]
class_names = ['bearish', 'bullish']
# model = SentimentClassifier(len(class_names))
# optimizer = AdamW(model.parameters(), lr=2e-5, correct_bias=False)
# device = torch.device('cpu')
# checkpoint = torch.load("./twitter_sentiment_model.pth", map_location=device)
# model.load_state_dict(checkpoint['model_state_dict'])
# optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
# epoch = checkpoint['epoch']
equities = []
datetimes = []

In [6]:

global equities, datetimes
twitter_client = twitter_api.TwitterClient()
tweet_preprocessor = twitter_api.TweetPreprocessor()
api = twitter_client.get_twitter_client_api()
latest_id = None
hash_tag_dict = dict.fromkeys(portfolio, None)
placeholder = st.empty()

AttributeError: module 'twitter_api' has no attribute 'TwitterClient'

In [5]:
tweet_preprocessor

NameError: name 'tweet_preprocessor' is not defined

In [6]:
from textblob import TextBlob


print(TextBlob('1: i love you').sentiment.polarity)
print(TextBlob('2: i do not love you').sentiment.polarity)
print(TextBlob('3: i hate you').sentiment.polarity)
print(TextBlob('4:go away').sentiment.polarity)

0.5
-0.25
-0.8
0.0
